Import Libraries

In [1]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 

Build New Model

In [2]:
from PIL import Image
import cv2
import torchvision.transforms as transforms
import collections
import random
from random import seed, choice, sample
import os
import json
from google.colab.patches import cv2_imshow
import sys
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False
from os.path import exists, join, basename, splitext
import csv
import glob

In [3]:
%matplotlib inline
import os
import sys
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import skimage.transform
from PIL import Image
from collections import Counter
from tqdm.notebook import tqdm

import torch
torch.cuda.empty_cache()
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu

import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from gensim.models.fasttext import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
os.chdir('/content/drive/My Drive/Final Year Project/')

In [7]:
def unique_list(words, confs):
  unique_list_words = []
  unique_list_confs = []
  for i in range(0, len(words)):
    if words[i] not in unique_list_words:
      unique_list_words.append(words[i])
      unique_list_confs.append(confs[i])
  return unique_list_words, unique_list_confs

In [8]:
print(os.path.abspath('.'))

/content/drive/My Drive/Final Year Project


In [9]:
filename = '/content/drive/My Drive/Final Year Project/input_images_train.txt'
read_file = open(filename,"r") 

train_image_paths = []
train_image_names = []
#print("Output of Read function is ")
data_read = read_file.read()
data_into_list = data_read.split("\n")
#print(data_into_list)
for i in range(len(data_into_list)):
    if(len(data_into_list[i])>1):
        train_image_paths.append(data_into_list[i])
        path = data_into_list[i]
        name_list = path.split("/")
        name = name_list[len(name_list)-1]
        #print(path)
        train_image_names.append(name)
print("No. of images: ",len(train_image_paths), len(train_image_names))

No. of images:  23431 23431


In [10]:
filename = '/content/drive/My Drive/Final Year Project/input_images_val.txt'
read_file = open(filename,"r") 

val_image_paths = []
val_image_names = []
#print("Output of Read function is ")
data_read = read_file.read()
data_into_list = data_read.split("\n")
#print(data_into_list)
for i in range(len(data_into_list)):
    if(len(data_into_list[i])>1):
        val_image_paths.append(data_into_list[i])
        path = data_into_list[i]
        name_list = path.split("/")
        name = name_list[len(name_list)-1]
        #print(path)
        val_image_names.append(name)
print("No. of images: ",len(val_image_paths), len(val_image_names))

No. of images:  7750 7750


In [11]:
# sets device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
image_paths = ['/content/drive/My Drive/Final Year Project/dataset/random/sample.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023939.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00000112.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023901.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00000799.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00011888.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00005499.jpg',
               '/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023410.jpg']
image_names = ['sample.jpg','VizWiz_train_00023939.jpg','VizWiz_train_00000112.jpg','VizWiz_train_00023901.jpg',
               'VizWiz_train_00000799.jpg','VizWiz_train_00011888.jpg','VizWiz_train_00005499.jpg','VizWiz_train_00023410.jpg']

#print(image_paths)

print(len(image_paths), len(image_names))

8 8


In [13]:
filename = '/content/drive/My Drive/Final Year Project/input_images.txt'
write_file = open(filename,"w+")
for i in range(len(image_paths)):
  img_path = image_paths[i] + '\n'
  write_file.write(img_path)
write_file.close()

In [14]:
git_repo_url = 'https://github.com/clovaai/CRAFT-pytorch.git'
project_name = splitext(basename(git_repo_url))[0]
sys.path.append(project_name)
if not exists(project_name):
  !git clone -q {git_repo_url}
  #!cd {project_name} && pip install -q -r requirements.txt
pretrained_model = 'craft_mlt_25k.pth'
if not exists(pretrained_model):
  !wget -q -O {pretrained_model} 'https://drive.google.com/uc?authuser=0&id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ&export=download'
os.chdir('/content/drive/My Drive/Final Year Project/')
print(project_name)
# verify CUDA
!/usr/local/cuda/bin/nvcc --version
!cd 'CRAFT-pytorch' && python pipeline.py --trained_model='/content/drive/My Drive/Final Year Project/craft_mlt_25k.pth' --file_name='/content/drive/My Drive/Final Year Project/input_images.txt'

CRAFT-pytorch
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
filename:  /content/drive/My Drive/Final Year Project/input_images.txt
No. of images:  8 8
in craft model for text detection
Loading weights from checkpoint (/content/drive/My Drive/Final Year Project/craft_mlt_25k.pth)
Available:  True
image bounding box found for:  /content/drive/My Drive/Final Year Project/dataset/random/sample.jpg
image bounding box found for:  /content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023939.jpg
image bounding box found for:  /content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00000112.jpg
image bounding box found for:  /content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023901.jpg
image bounding box found for:  /content/drive/My Drive/Final Year Project/dat

In [15]:
!cd 'CRAFT-pytorch' && python crop_image.py

/data/ not using it
in crop images function
/content/drive/My Drive/Final Year Project/dataset/random/sample.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023939.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00000112.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023901.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00000799.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00011888.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00005499.jpg
/content/drive/My Drive/Final Year Project/dataset/train-test-ocr/VizWiz_train_00023410.jpg
the text on the images are cropped


In [16]:
git_repo_url_pred = 'https://github.com/clovaai/deep-text-recognition-benchmark.git'
project_name_pred = splitext(basename(git_repo_url_pred))[0]
print(project_name_pred)
if not exists(project_name_pred):
  # clone and install
  !git clone -q {git_repo_url_pred}
  #!cd {project_name_pred} && pip install -q -r requirements.txt
!cd 'CRAFT-pytorch' && python predict.py \
    --Transformation None --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction CTC \
    --image_folder '/content/drive/My Drive/Final Year Project/CRAFT-pytorch/cropped_words' \
    --saved_model '/content/drive/My Drive/Final Year Project/None-VGG-BiLSTM-CTC.pth'

deep-text-recognition-benchmark
No Transformation module specified
model input parameters 32 100 20 1 512 256 37 25 None VGG BiLSTM CTC
loading pretrained model from /content/drive/My Drive/Final Year Project/None-VGG-BiLSTM-CTC.pth
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
--------------------------------------------------------------------------------
image_path               	 predicted_labels         	 confidence score
--------------------------------------------------------------------------------
VizWiz_train_00000112_709.2761_1531.7844_1204.0363_1537.7455_1202.4902_1666.05

In [17]:
#-----OCR--------
def ocr():       
    csv_dict = collections.defaultdict(list)

    with open("/content/drive/My Drive/Final Year Project/CRAFT-pytorch/data.csv", "r") as source_data:
        csvreader = csv.reader(source_data)
        header = next(csvreader)
        rows = []
        for row in csvreader:
          print(row)
          rows.append(row)

    with open("/content/drive/My Drive/Final Year Project/CRAFT-pytorch/words_ocr.csv", "r") as source:
        reader = csv.reader(source)
        header = next(reader)
        for key,*rest in reader:
            #print(key)
            csv_dict[key] += filter(None,rest)
    print(csv_dict.keys())
        

    keys=[]
    for i in range(len(rows)):
      row=rows[i]
      keys.append(row[1])
    print("keys len: ",len(keys))

    key_array=[]
    word_array=[]
    conf_array=[]
    countd=0

    for i in range(len(keys)):
      key = keys[i]
      if key in csv_dict.keys():
        value = csv_dict[key]
        key_array.append(key)
        word=[]
        conf=[]
        for i in range(1, len(value), 3):
          #print(value[i], value[i+1])
          word.append(value[i])
          conf.append(value[i+1])
        word_array.append(word)
        conf_array.append(conf)
      else:
        key_array.append(row[1])
        word_array.append([])
        conf_array.append([])

    print(len(key_array), len(word_array), len(conf_array))
    #print(conf_array)
    print(word_array)
   

    return word_array

In [18]:
word_array_ocr = ocr()

['0', 'sample.jpg', '/content/drive/My Drive/Final Year Project/dataset/random/sample.jpg', "{'0.9526028': array([[ 48.      ,  16.      ],\n       [122.666664,  16.      ],\n       [122.666664,  28.      ],\n       [ 48.      ,  28.      ]], dtype=float32), '0.9407104': array([[136.     ,  16.     ],\n       [197.33333,  16.     ],\n       [197.33333,  28.     ],\n       [136.     ,  28.     ]], dtype=float32), '0.92722774': array([[262.66666,  16.     ],\n       [309.33334,  16.     ],\n       [309.33334,  28.     ],\n       [262.66666,  28.     ]], dtype=float32), '0.9265149': array([[322.66666 ,  16.      ],\n       [352.      ,  16.      ],\n       [352.      ,  29.333334],\n       [322.66666 ,  29.333334]], dtype=float32), '0.94583446': array([[366.66666,  16.     ],\n       [412.     ,  16.     ],\n       [412.     ,  28.     ],\n       [366.66666,  28.     ]], dtype=float32), '0.79670876': array([[225.33333 ,  24.      ],\n       [236.      ,  24.      ],\n       [236.      ,  

In [19]:
print("word array len: ",len(word_array_ocr))
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2

fast_Text_model = FastText(word_array_ocr,
                      size=embedding_size,
                      window=window_size,
                      min_count=1,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)
print(fast_Text_model)

features_ocr=[]
for i in range(len(word_array_ocr)):
  ft1=np.empty((300))
  for word in word_array_ocr[i]:
    ft1+=fast_Text_model.wv[word]
  #print(ft1.shape)
  #print(ft1)
  features_ocr.append(ft1)
    
for i in range(len(features_ocr)):
  ft1 = features_ocr[i]
  ft1 = ft1.reshape(1,300)
  ft1_out = np.tile(ft1, (14,14,1,1))
  #print(ft1_out.shape)
  ft1_out = np.transpose(ft1_out, [2, 3, 0, 1])
  #print(ft1_out.shape)
  features_ocr[i] = ft1_out

print(len(features_ocr), features_ocr[0].shape)

word array len:  8
FastText(vocab=128, size=300, alpha=0.025)
8 (1, 300, 14, 14)
